<a href="https://colab.research.google.com/github/BetriuJaume/Projecte_Prog/blob/master/RandomForest_Code_and_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyreadstat
import pyreadstat

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.utils import resample
from imblearn.over_sampling import SMOTENC
!pip install h2o
import h2o
from h2o.estimators import H2ORandomForestEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,57 mins 11 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.7
H2O_cluster_version_age:,1 month and 4 days
H2O_cluster_name:,H2O_from_python_unknownUser_bojpcy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.166 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [32]:
dades=pd.read_spss('EarlyLifeCovidPACIENTES V6 ID.sav')

def canvi_tab(elem):
  if(elem=='si (pasado o actual)'):
    return 'yes'
  if(elem=='nunca'):
    return 'no'
def canvi(elem):
  if(elem=='si'):
    return 'yes'
  if(elem=='no'):
    return 'no'
def canvi_UCI(elem):
  if(elem==1):
    return 'Yes'
  if(elem==0):
    return 'No'
def canvi_sex(elem):
  if(elem=='mujer'):
    return 'woman'
  if(elem=='hombre'):
    return 'man'

dades['Tabaco_SIno']=dades['Tabaco_SIno'].apply(canvi_tab)
dades['UCI']=dades['UCI'].apply(canvi_UCI)
for columna in dades.columns[14:]:
  dades[columna]=dades[columna].apply(canvi)
dades['Gender']=dades['Gender'].apply(canvi_sex)
dades['IUGR_calc']=dades['IUGR_calc'].apply(canvi_UCI)
dades.columns=pd.Index(['ID', 'UCI', 'IUGR_missing', 'Age', 'Gender', 'Size', 'Weight',
       'Size_mt', 'IMC', 'BW', 'BW_2500', 'percentil_birth', 'IUGR_calc',
       'Tobacco_yes_no', 'Hipertension', 'Heart_diseases', 'DM', 'Dyslipidemia',
       'Obesity', 'Kidney_disease', 'Autoimmune', 'Cancer', 'Thyroid', 'Infectious',
       'Psychiatric'])

In [33]:
dades_train = dades.sample(frac=0.8, random_state=25).drop('ID',axis=1)
dades_test = dades.drop(dades_train.index).drop('ID',axis=1)

We will design a function to caluculate the accuraccy of the models:

In [34]:
def precisions(reals,prediccions):
  cont_Yes=0
  cont_No=0
  for i in range(len(reals)):
    if (reals.iloc[i]==prediccions[i]) & (reals.iloc[i]=='Yes'):
      cont_Yes=cont_Yes+1
    if (reals.iloc[i]==prediccions[i]) & (reals.iloc[i]=='No'):
      cont_No=cont_No+1
  print('Precision for the Yes: '+str(cont_Yes/len(reals[reals=='Yes'])))
  print('Precision for the No: '+str(cont_No/len(reals[reals=='No'])))
  print('Total precision: '+str((cont_Yes+cont_No)/len(reals)))

Here we will create the upsampled training data to balance the dataset:

In [35]:
dades_train_UCI_No=dades_train[dades_train['UCI']=='No']
dades_train_UCI_Yes=dades_train[dades_train['UCI']=='Yes']
dades_UCI_Yes_upsampled=resample(dades_train_UCI_Yes,replace=True,n_samples=len(dades_train_UCI_No),random_state=123)
dades_train_upsampled=pd.concat([dades_train_UCI_No,dades_UCI_Yes_upsampled])

 and here we will create the training data balanced with SMOTENC

In [36]:
categ_indexes=[0,2,7,8]+list(range(10,dades_train.shape[1]-1))
smot = SMOTENC(categorical_features=categ_indexes, random_state=123)
dades_train_SMOTENC, dades_train_SMOTENC_UCI = smot.fit_resample(dades_train.drop('UCI',axis=1), dades_train['UCI'])
dades_train_SMOTENC['UCI']=dades_train_SMOTENC_UCI

#Training of the Random Forest models from the library H2O

We will use the upsampling training dataset for training one model, the SMOTENC training dataset for training another model and then we will compare them

In [37]:
dades_train_upsampled_h2o=h2o.H2OFrame(dades_train_upsampled)
dades_train_SMOTENC_h2o=h2o.H2OFrame(dades_train_SMOTENC)
predictors_upsampling=['BW_2500','Gender','Hipertension','Obesity','Age','IMC']   #h2o.H2OFrame(dades_train_upsampled).drop('UCI',axis=1).columns
predictors_SMOTENC=['BW_2500','Cancer','DM','Dyslipidemia','Gender','Hipertension','Obesity','Psychiatric','Age']
response='UCI'

model_random_forest_upsampling = H2ORandomForestEstimator(ntrees=500,
                                    max_depth=20,
                                    min_rows=10,
                                    seed=123,
                                    mtries=-1)
model_random_forest_SMOTENC = H2ORandomForestEstimator(ntrees=500,
                                    max_depth=20,
                                    min_rows=10,
                                    seed=123,
                                    mtries=-1)

model_random_forest_upsampling=model_random_forest_upsampling.train(x=predictors_upsampling,
                                                         y=response,
                                                         training_frame=dades_train_upsampled_h2o)
model_random_forest_SMOTENC=model_random_forest_SMOTENC.train(x=predictors_SMOTENC,
                                                      y=response,
                                                      training_frame=dades_train_SMOTENC_h2o)

prediccions_random_forest_upsampling=model_random_forest_upsampling.predict(h2o.H2OFrame(dades_test).drop('UCI',axis=1))
prediccions_random_forest_SMOTENC=model_random_forest_SMOTENC.predict(h2o.H2OFrame(dades_test).drop('UCI',axis=1))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


Once we have both models trained we can calculate our predictions

In [38]:
prediccions_random_forest_upsampling=model_random_forest_upsampling.predict(h2o.H2OFrame(dades_test).drop('UCI',axis=1))
prediccions_random_forest_SMOTENC=model_random_forest_SMOTENC.predict(h2o.H2OFrame(dades_test).drop('UCI',axis=1))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


And check the accuracy

In [39]:
prediccions_random_forest_upsampling= prediccions_random_forest_upsampling['predict'].as_data_frame()
prediccions_random_forest_upsampling=prediccions_random_forest_upsampling['predict'].tolist()

prediccions_random_forest_SMOTENC= prediccions_random_forest_SMOTENC['predict'].as_data_frame()
prediccions_random_forest_SMOTENC=prediccions_random_forest_SMOTENC['predict'].tolist()

print('Upsampling:')
precisions(dades_test['UCI'],prediccions_random_forest_upsampling)
print('___________________________________')
print('SMOTENC:')
precisions(dades_test['UCI'],prediccions_random_forest_SMOTENC)

Upsampling:
Precision for the Yes: 0.7272727272727273
Precision for the No: 0.7647058823529411
Total precision: 0.759493670886076
___________________________________
SMOTENC:
Precision for the Yes: 0.7272727272727273
Precision for the No: 0.6323529411764706
Total precision: 0.6455696202531646


#Design, training and hiperparameter adjusting of the Neural Network

In [40]:
import torch
import torch.utils.data as data_utils
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

We have to do a first codification of the categorical predictors because Torch tensors only work with categorical values

In [41]:
def codGender(elem):
  if elem=='man':
    return 1.
  else:
    return 0.
def codUCI(elem):
  if elem=='Yes':
    return 1.
  else:
    return 0.
def codBW(elem):
  if elem=='BW normal':
    return 0.
  else:
    return 1.
def cod(elem):
  if elem=='yes':
    return 1.
  else:
    return 0.

dades_cod_train=dades_train_upsampled.drop('IUGR_missing',axis=1).copy()
dades_cod_test=dades_test.drop('IUGR_missing',axis=1).copy()

for df in [dades_cod_train,dades_cod_test]:
  df['UCI']=df['UCI'].apply(codUCI)
  df['Gender']=df['Gender'].apply(codGender)
  df['BW_2500']=df['BW_2500'].apply(codBW)
  df['IUGR_calc']=df['IUGR_calc'].apply(codUCI)
  for element in df.columns[11:]:
    df[element]=df[element].apply(cod)

target_train=torch.tensor(dades_cod_train['UCI'].values)
features_train=torch.tensor(dades_cod_train.drop('UCI',axis=1).values)
Dades_Tensor_train=TensorDataset(features_train,target_train)

target_test=torch.tensor(dades_cod_test['UCI'].values)
features_test=torch.tensor(dades_cod_test.drop('UCI',axis=1).values)
Dades_Tensor_test=TensorDataset(features_test,target_test)

from torch.utils.data import random_split

train_torch,val_torch=random_split(Dades_Tensor_train,[int(0.8*len(Dades_Tensor_train))+1,int(0.2*len(Dades_Tensor_train))],generator=torch.Generator().manual_seed(123))
test_torch=Dades_Tensor_test

train_torch1=DataLoader(train_torch,batch_size=10,shuffle=True)
val_torch1=DataLoader(val_torch,batch_size=5,shuffle=True)
test_torch1=DataLoader(test_torch,batch_size=5,shuffle=True)

We design our net

In [42]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.input_layer=nn.Linear(features_train.size()[1],14) #weights
    #self.hidden1=nn.Linear(14,7)
    #self.hidden2=nn.Linear(7,7)
    #self.hidden3=nn.Linear(7,10)
    #self.hidden4=nn.Linear(10,7)
    self.output=nn.Linear(14,2)
  
  def forward(self, data): #activation functions, we will use the ReLu function
    data=F.relu(self.input_layer(data))
    #data=F.relu(self.hidden1(data))
    #data=F.relu(self.hidden2(data))
    #data=F.relu(self.hidden3(data))
    #data=F.relu(self.hidden4(data))
    data=self.output(data)
    return F.log_softmax(data, dim=1)

 and we train it with the **upsampling training dataset** and once we have the NN trained we will use the validation dataset to adjust the hiperparameter **rep**:

In [43]:
Accuracy_yes=[]
Accuracy_no=[]

network=Network()
learn_rate=optim.Adam(network.parameters(), lr=0.0001)
for rep in range(30):
  for i in range(rep):
    for data in train_torch1:
      predictors, objectiu=data
      network.zero_grad()
      result=network(predictors.float())
      loss=F.nll_loss(result,objectiu.long())  #loss=F.cross_entropy(result,objectiu.long())  # negative log likelihood loss
      loss.backward()
      learn_rate.step()
    #print(loss)

  network.eval() #we tell the network that we are not training anymore

  correct_No=0
  correct_Yes=0
  total_No=0
  total_Yes=0

  with torch.no_grad(): #validation set to adjust the hiperparameters
    for data in val_torch1:
      predictors, objectiu=data
      result=network(predictors.float())
      for index, tensor_value in enumerate(result):
        if(objectiu[index]==1):
          total_Yes=total_Yes+1
          if(torch.argmax(tensor_value)==objectiu[index]):
            correct_Yes=correct_Yes+1
        if(objectiu[index]==0):
          total_No=total_No+1
          if(torch.argmax(tensor_value)==objectiu[index]):
            correct_No=correct_No+1

  accuracy_Yes=correct_Yes/total_Yes
  accuracy_No=correct_No/total_No

  Accuracy_yes.append(accuracy_Yes)
  Accuracy_no.append(accuracy_No)

  print('Accuracy Yes '+str(accuracy_Yes)+' with rep='+str(rep))
  print('Accuracy No '+str(accuracy_No)+' with rep='+str(rep))
  print('_________________________________________________________')

Accuracy Yes 0.0 with rep=0
Accuracy No 1.0 with rep=0
_________________________________________________________
Accuracy Yes 0.0 with rep=1
Accuracy No 1.0 with rep=1
_________________________________________________________
Accuracy Yes 0.0 with rep=2
Accuracy No 1.0 with rep=2
_________________________________________________________
Accuracy Yes 0.0 with rep=3
Accuracy No 1.0 with rep=3
_________________________________________________________
Accuracy Yes 0.423728813559322 with rep=4
Accuracy No 0.5416666666666666 with rep=4
_________________________________________________________
Accuracy Yes 0.4067796610169492 with rep=5
Accuracy No 0.625 with rep=5
_________________________________________________________
Accuracy Yes 0.22033898305084745 with rep=6
Accuracy No 0.7291666666666666 with rep=6
_________________________________________________________
Accuracy Yes 0.3898305084745763 with rep=7
Accuracy No 0.6666666666666666 with rep=7
_______________________________________________

Now we will train our NN with the **SMOTENC training dataset**. First we make the codification of the categorical values as we did with the other one:

In [44]:
first_col=dades_train_SMOTENC.pop('UCI')
dades_train_SMOTENC.insert(0, 'UCI', first_col)
dades_train_SMOTENC=dades_train_SMOTENC.sample(frac=1)

dades_cod_train=dades_train_SMOTENC.drop('IUGR_missing',axis=1).copy()
dades_cod_test=dades_test.drop('IUGR_missing',axis=1).copy()

for df in [dades_cod_train,dades_cod_test]:
  df['UCI']=df['UCI'].apply(codUCI)
  df['Gender']=df['Gender'].apply(codGender)
  df['BW_2500']=df['BW_2500'].apply(codBW)
  df['IUGR_calc']=df['IUGR_calc'].apply(codUCI)
  for element in df.columns[11:]:
    df[element]=df[element].apply(cod)

target_train=torch.tensor(dades_cod_train['UCI'].values)
features_train=torch.tensor(dades_cod_train.drop('UCI',axis=1).values)
Dades_Tensor_train=TensorDataset(features_train,target_train)

target_test=torch.tensor(dades_cod_test['UCI'].values)
features_test=torch.tensor(dades_cod_test.drop('UCI',axis=1).values)
Dades_Tensor_test=TensorDataset(features_test,target_test)

from torch.utils.data import random_split
#train_torch,val_torch,test_torch=random_split(Dades_Tensor,[int(0.7*len(dades))+2,int(0.15*len(dades)),int(0.15*len(dades))],generator=torch.Generator().manual_seed(123))

train_torch,val_torch=random_split(Dades_Tensor_train,[int(0.8*len(Dades_Tensor_train))+1,int(0.2*len(Dades_Tensor_train))],generator=torch.Generator().manual_seed(22))
test_torch=Dades_Tensor_test

train_torch1=DataLoader(train_torch,batch_size=10,shuffle=True)
val_torch1=DataLoader(val_torch,batch_size=10,shuffle=True)
test_torch1=DataLoader(test_torch,batch_size=10,shuffle=True)

and we train and validate the Neural Network:

In [45]:
Accuracy_yes=[]
Accuracy_no=[]

network=Network()
learn_rate=optim.Adam(network.parameters(), lr=0.0001)
for rep in range(30):
  for i in range(rep):
    for data in train_torch1:
      predictors, objectiu=data
      network.zero_grad()
      result=network(predictors.float())
      loss=F.nll_loss(result,objectiu.long())  #loss=F.cross_entropy(result,objectiu.long())  # negative log likelihood loss
      loss.backward()
      learn_rate.step()
    #print(loss)

  network.eval() #we tell the network that we are not training anymore

  correct_No=0
  correct_Yes=0
  total_No=0
  total_Yes=0

  with torch.no_grad(): #validation set to adjust the hiperparameters
    for data in val_torch1:
      predictors, objectiu=data
      result=network(predictors.float())
      for index, tensor_value in enumerate(result):
        if(objectiu[index]==1):
          total_Yes=total_Yes+1
          if(torch.argmax(tensor_value)==objectiu[index]):
            correct_Yes=correct_Yes+1
        if(objectiu[index]==0):
          total_No=total_No+1
          if(torch.argmax(tensor_value)==objectiu[index]):
            correct_No=correct_No+1

  accuracy_Yes=correct_Yes/total_Yes
  accuracy_No=correct_No/total_No

  Accuracy_yes.append(accuracy_Yes)
  Accuracy_no.append(accuracy_No)

  print('Accuracy Yes '+str(accuracy_Yes)+' with rep='+str(rep))
  print('Accuracy No '+str(accuracy_No)+' with rep='+str(rep))
  print('_________________________________________________________')

Accuracy Yes 1.0 with rep=0
Accuracy No 0.0 with rep=0
_________________________________________________________
Accuracy Yes 1.0 with rep=1
Accuracy No 0.0 with rep=1
_________________________________________________________
Accuracy Yes 1.0 with rep=2
Accuracy No 0.0 with rep=2
_________________________________________________________
Accuracy Yes 1.0 with rep=3
Accuracy No 0.0 with rep=3
_________________________________________________________
Accuracy Yes 0.5777777777777777 with rep=4
Accuracy No 0.27419354838709675 with rep=4
_________________________________________________________
Accuracy Yes 0.5555555555555556 with rep=5
Accuracy No 0.3064516129032258 with rep=5
_________________________________________________________
Accuracy Yes 0.5333333333333333 with rep=6
Accuracy No 0.3709677419354839 with rep=6
_________________________________________________________
Accuracy Yes 0.3333333333333333 with rep=7
Accuracy No 0.6129032258064516 with rep=7
_________________________________

Finally we will use te neural network trained by SMOTENC sample in the test dataset beacuse is the one that works better with the hyperparameter **rep**=29:

In [47]:
correct_No=0
correct_Yes=0
total_No=0
total_Yes=0
rep=25
for data in test_torch1:
  predictors, objectiu=data
  result=network(predictors.float())
  for index, tensor_value in enumerate(result):
    if(objectiu[index]==1):
      total_Yes=total_Yes+1
      if(torch.argmax(tensor_value)==objectiu[index]):
        correct_Yes=correct_Yes+1

    if(objectiu[index]==0):
      total_No=total_No+1
      if(torch.argmax(tensor_value)==objectiu[index]):
        correct_No=correct_No+1

accuracy_Yes=correct_Yes/total_Yes
accuracy_No=correct_No/total_No

Accuracy_yes.append(accuracy_Yes)
Accuracy_no.append(accuracy_No)

print('Accuracy Yes '+str(accuracy_Yes)+' with rep='+str(rep))
print('Accuracy No '+str(accuracy_No)+' with rep='+str(rep))
print('_________________________________________________________')


Accuracy Yes 0.0 with rep=25
Accuracy No 0.9117647058823529 with rep=25
_________________________________________________________
